In [ ]:
# pip install kagglehub

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("shubhambathwal/flight-price-prediction")

# print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from shapely.geometry import Point
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import *

In [ ]:
data=pd.read_csv(r"C:\Users\segev\code_notebooks\Clean_Dataset.csv")

In [ ]:
(data.price*0.04).describe()

In [ ]:
123071

In [ ]:
data.info()

In [ ]:
pd.DataFrame(data.airline.value_counts())

In [ ]:
len(set(data.flight))

In [ ]:
pd.DataFrame(data.source_city.value_counts())

In [ ]:
pd.DataFrame(data.departure_time.value_counts())

In [ ]:
pd.DataFrame(data.arrival_time.value_counts())

In [ ]:
pd.DataFrame(data.stops.value_counts())

In [ ]:
pd.DataFrame(data.destination_city.value_counts())  


In [ ]:
# pip install contextily


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
from shapely.geometry import Point

df = pd.DataFrame({'destination_city': ['Mumbai', 'Delhi', 'Bangalore', 'Kolkata', 'Hyderabad', 'Chennai']})

df = df.drop_duplicates(subset=['destination_city'])

geolocator = Nominatim(user_agent="geo_plotting", timeout=10)  # Increased timeout

def get_coordinates_and_state(city):
    retries = 5
    for i in range(retries):
        try:
            location = geolocator.geocode(city, timeout=10)  # Increase timeout
            if location:
                address = location.raw.get('address', {})
                state = address.get('state')  # Get the state name
                return location.latitude, location.longitude, state
        except (GeocoderTimedOut, GeocoderServiceError):
            print(f"Retry {i+1}/{retries} for {city}...")
            time.sleep(2)  # Wait before retrying
    return None, None, None  # Return None if all retries fail

# Apply geocoding to get coordinates and state
df['latitude'], df['longitude'], df['state'] = zip(*df['destination_city'].apply(get_coordinates_and_state))

# Drop rows where geocoding failed
df = df.dropna(subset=['latitude', 'longitude'])

# Convert to GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326").to_crs(epsg=3857)

# Plot
fig, ax = plt.subplots(figsize=(14, 14))
gdf.plot(ax=ax, color='red', alpha=0.6, edgecolor='k', markersize=10)  # Fixed size for markers

# Add OpenStreetMap basemap
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

# Annotate city names and states
for x, y, label, state in zip(gdf.geometry.x, gdf.geometry.y, gdf['destination_city'], gdf['state']):
    ax.text(x, y, f"{label}, {state}", fontsize=9, ha='right', color='black', weight='bold')

# Remove axis for better visualization
ax.set_axis_off()

plt.show()


In [ ]:
pd.DataFrame(data['duration'].describe())

In [ ]:
pd.DataFrame(data['days_left'].describe())

In [ ]:
pd.DataFrame(data['price'].describe())

In [ ]:
len(set(data['price'])), data.shape

In [ ]:
pd.DataFrame(data['class'].value_counts())

# PreProcessing

In [ ]:
data.columns 

In [ ]:
data=data.drop(columns=['Unnamed: 0', 'flight'])

In [ ]:
data.isnull().sum()

In [ ]:
data.sample()

In [ ]:
raw=data.copy()

In [ ]:
set(raw['class'])

In [ ]:
raw['class']=raw['class'].replace({'Business':1,
                                   'Economy':0})

In [ ]:
set(raw['class'])

In [ ]:
raw.stops=pd.factorize(raw.stops)[0]

In [ ]:
set(raw.stops)

In [ ]:
raw=raw.join(pd.get_dummies(raw.airline, prefix='airline')).drop(columns=['airline'])

raw=raw.join(pd.get_dummies(raw.source_city, prefix='source_city')).drop(columns=['source_city'])

raw=raw.join(pd.get_dummies(raw.destination_city, prefix='destination_city')).drop(columns=['destination_city'])

raw=raw.join(pd.get_dummies(raw.arrival_time, prefix='arrival_time')).drop(columns=['arrival_time'])

raw=raw.join(pd.get_dummies(raw.departure_time, prefix='departure_time')).drop(columns=['departure_time'])



In [ ]:
raw.sample()

# Train regression model

**RandomForestRegressor is a supervised machine learning algorithm used for predicting continuous numerical values by averaging the outputs of multiple decision trees.**

**Each tree is trained on a random subset of the data and features—a process known as bagging—which introduces diversity and reduces overfitting. The final prediction is obtained by aggregating the individual predictions from all trees, typically through averaging, resulting in improved accuracy and robustness.**

**This ensemble approach allows RandomForestRegressor to capture complex, non-linear relationships in data without requiring extensive preprocessing or feature scaling. It is widely used in various applications, including stock price forecasting, real estate valuation, and time series prediction.**

https://www.youtube.com/watch?v=X1MRbEnEq2s&ab_channel=SuperDataScience  V 

https://www.youtube.com/watch?v=YUsx5ZNlYWc&ab_channel=Ryan%26MattDataScience  X


In [ ]:
X = raw.drop(columns='price')

y= raw.price

In [ ]:
X_train ,X_test , y_train , y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
reg= RandomForestRegressor(n_jobs=-1)


In [ ]:
reg.fit(X_train, y_train)

In [ ]:
reg

​The reg.score(X, y) method in RandomForestRegressor computes the R² score (coefficient of determination), indicating how well the model's predictions match the actual target values. An R² of 1.0 signifies perfect predictions, 0.0 means the model performs no better than predicting the mean of the target values, and negative values indicate worse performance than this baseline. This metric provides a quick assessment of model performance, but for a comprehensive evaluation, consider additional metrics like Mean Absolute Error (MAE) or Root Mean Squared Error (RMSE).

In [ ]:
reg.score(X_test, y_test)

An R² score of 0.98527 from reg.score(X_test, y_test) indicates that your RandomForestRegressor model explains approximately 98.5% of the variance in the test data. This suggests a strong fit between the model's predictions and the actual target values.​

In [ ]:
from sklearn.metrics import *

The from sklearn.metrics import * statement imports all available functions and classes from the sklearn.metrics module, which provides a comprehensive suite of tools for evaluating machine learning models. These tools include:​

Classification Metrics: Functions like accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, and confusion_matrix are used to assess the performance of classification models.​

Regression Metrics: Functions such as mean_squared_error, mean_absolute_error, r2_score, and explained_variance_score help evaluate regression models.​

Clustering Metrics: Metrics like adjusted_rand_score, normalized_mutual_info_score, and silhouette_score are used to assess clustering algorithms.​

Pairwise Metrics: Functions like pairwise_distances and pairwise_kernels compute distances or similarities between pairs of samples.​

Utility Functions: Tools such as make_scorer and get_scorer assist in creating custom scoring functions and retrieving predefined scorers.​

In [ ]:
import math

In [ ]:
y_pred= reg.predict(X_test)

In [ ]:
print(f"R2: {r2_score(y_test, y_pred):.3f}")

print(f"MAE: {mean_absolute_error(y_test, y_pred):.3f}")

print(f"MSE: {mean_squared_error(y_test, y_pred):.3f}")

print(f"R-MSE: {math.sqrt(mean_squared_error(y_test, y_pred)):.3f}")

The model demonstrates high predictive accuracy, as evidenced by the R² value close to 1. 

The MAE and RMSE values suggest that, on average, predictions are within a reasonable range of the actual prices.

However, the relatively higher RMSE compared to MAE indicates the presence of some larger errors, which could be due to outliers or specific instances where the model's predictions deviate more significantly.

In [ ]:

residuals = y_test - y_pred

plt.figure(figsize=(12, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel("Predicted Flight Price", fontsize=14)
plt.ylabel("Residual (Actual - Predicted)", fontsize=14)
plt.title("Residual Plot: Predicted vs. Residuals", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True)
plt.show()


Centered residuals: The residuals are generally centered around the 0 line, which is what we expect in a good model.

No clear linear pattern: That's a sign your model isn't systematically under- or over-predicting across all price ranges.

Densely packed residuals: Many points are close to the zero line, indicating overall strong performance.



In [ ]:
from termcolor import colored

print(colored("Meta data regarding the predicted value:", color="cyan", attrs=["bold"]))

pd.DataFrame(raw.price.describe())

In [ ]:
len(y_pred),len(y_test)

In [ ]:


labels = [f"({np.array(y_test)[i]:.2f}, {np.array(y_pred)[i]:.2f})" for i in range(len(y_test))]

np.random.seed(42) 
indices_to_label = np.random.choice(len(y_test), size=5, replace=False)

plt.figure(figsize=(12, 8))
plt.scatter(np.array(y_test), y_pred, s=100)

plt.xlabel("Actual flight price", fontsize=14)
plt.ylabel("Predicted flight price", fontsize=14)
plt.title("Actual vs Predicted Flight Prices", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

for i in indices_to_label:
    plt.text(np.array(y_test)[i], np.array(y_pred)[i], labels[i],
             fontsize=10, ha='right', va='bottom')

plt.show()


In [ ]:
importances = reg.feature_importances_
feature_names = X_train.columns

feat_imp_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

feat_imp_df.head(5)

In [ ]:
plt.figure(figsize=(12, 6))
bars = plt.barh(feat_imp_df["Feature"].head(5),
                feat_imp_df["Importance"].head(5),
                color='skyblue')

plt.xlabel("Feature Importance Score")
plt.title("Feature Importances from RandomForestRegressor")
plt.gca().invert_yaxis() 
plt.grid(True, linestyle="--", alpha=0.9)

# Add value labels next to bars
for bar in bars:
    width = bar.get_width()
    plt.text(width + 0.001,              
             bar.get_y() + bar.get_height() / 2,
             f"{width:.3f}",         
             va='center', fontsize=10)

plt.tight_layout()
plt.show()


In [ ]:
top_features = feat_imp_df["Feature"].head(5)


print("\nTop 5 Feature Summary:\n")



pd.DataFrame({
    "Feature": top_features,
    "Min": X_train[top_features].min().values,
    "Max": X_train[top_features].max().values,
    "Unique Values": X_train[top_features].nunique().values
})

In [ ]:
sample_10_p=X_train.shape[0]*0.25

sample_10_p=round(sample_10_p, 0)

sample_10_p = int(sample_10_p)

In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import *

from scipy.stats  import *

In [ ]:


reg = RandomForestRegressor(n_jobs=-1, random_state=42)

param_grid = {
    'n_estimators': [50],         # Lower number of trees
    'max_depth': [10],            # Only one depth
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['sqrt']
}

grid_search = GridSearchCV(
    reg,
    param_grid,
    cv=2,            # Only 2-fold CV
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train[top_features][:sample_10_p], y_train[:sample_10_p])

print("Best Params:", grid_search.best_params_)

best_regressor= grid_search.best_estimator_



Sure! Here's a clear explanation of what the code does, in words only:

This script performs a quick hyperparameter tuning of a `RandomForestRegressor` model using only the top 5 most important features. 

It sets up a simple grid of parameters with minimal options to reduce computation time. The tuning is done using `GridSearchCV` with 2-fold cross-validation and only 10% of the training data. After fitting the model on the selected features and data subset, 

it retrieves the best-performing estimator from the grid search. This process helps in efficiently finding a decent model without consuming too many resources.

In [ ]:
param_grid

In [ ]:
grid_search

In [ ]:
best_regressor

In [ ]:
best_regressor.score(X_test[top_features][:sample_10_p], y_test[:sample_10_p])



In [ ]:
y_pred= best_regressor.predict(X_test[top_features])

In [ ]:
print(f"R2: {r2_score(y_test, y_pred):.3f}")

print(f"MAE: {mean_absolute_error(y_test, y_pred):.3f}")

print(f"MSE: {mean_squared_error(y_test, y_pred):.3f}")

print(f"R-MSE: {math.sqrt(mean_squared_error(y_test, y_pred)):.3f}") 



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=raw.price)
plt.show()


In [ ]:
from scipy.stats import zscore

z_scores = zscore(raw.price)
outliers = raw[abs(z_scores) > 3]


In [ ]:
outliers

In [ ]:
import matplotlib.pyplot as plt

residuals = y_test - y_pred

sns.histplot(residuals, bins=30, kde=True)
plt.title("Distribution of Residuals")
plt.show()

sns.boxplot(x=residuals)
plt.title("Outliers in Residuals")
plt.show()


In [ ]:
sample_y_test = np.array(y_test[:sample_10_p])
sample_y_pred = np.array(y_pred[:sample_10_p])

labels = [f"({sample_y_test[i]:.2f}, {sample_y_pred[i]:.2f})" for i in range(sample_10_p)]
    
np.random.seed(42)
indices_to_label = np.random.choice(sample_10_p, size=5, replace=False)

plt.figure(figsize=(12, 8))
plt.scatter(sample_y_test, sample_y_pred, s=100)

plt.xlabel("Actual flight price", fontsize=14)
plt.ylabel("Predicted flight price", fontsize=14)
plt.title("Actual vs Predicted Flight Prices", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

for i in indices_to_label:
    plt.text(sample_y_test[i], sample_y_pred[i], labels[i], fontsize=10, ha='right', va='bottom')

plt.show()


In [ ]:

# Assume y_train is a Pandas Series
Q1 = y_train.quantile(0.25)
Q3 = y_train.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter indices where target is within bounds
mask = (y_train >= lower_bound) & (y_train <= upper_bound)

# Apply mask to X_train and y_train
X_train_clean = X_train[top_features].iloc[:sample_10_p][mask]
y_train_clean = y_train.iloc[:sample_10_p][mask]


In [ ]:
reg = RandomForestRegressor(n_jobs=-1, random_state=42)

param_grid = {
    'n_estimators': [50],
    'max_depth': [10],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'max_features': ['sqrt']
}

grid_search = GridSearchCV(
    reg,
    param_grid,
    cv=2,
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_clean, y_train_clean)

print("Best Params:", grid_search.best_params_)
best_regressor = grid_search.best_estimator_


In [ ]:

y_pred = best_regressor.predict(X_test[top_features])

# Evaluate
print(f"R2: {r2_score(y_test, y_pred):.3f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.3f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.3f}")
print(f"R-MSE: {math.sqrt(mean_squared_error(y_test, y_pred)):.3f}")


In [ ]:

sample_y_test = np.array(y_test[:sample_10_p])
sample_y_pred = np.array(y_pred[:sample_10_p])

labels = [f"({sample_y_test[i]:.2f}, {sample_y_pred[i]:.2f})" for i in range(sample_10_p)]

np.random.seed(42)
indices_to_label = np.random.choice(sample_10_p, size=5, replace=False)

plt.figure(figsize=(12, 8))
plt.scatter(sample_y_test, sample_y_pred, s=100, alpha=0.6, edgecolor='k')

plt.xlabel("Actual Flight Price", fontsize=14)
plt.ylabel("Predicted Flight Price", fontsize=14)
plt.title("Actual vs Predicted Flight Prices", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

for i in indices_to_label:
    plt.text(sample_y_test[i], sample_y_pred[i], labels[i],
             fontsize=10, ha='right', va='bottom', color='darkblue')

plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()
